In [1]:
from os import listdir
import numpy as np
import pandas as pd

In [2]:
trainf=list()
trainfe=list()
validf=list()
validfe=list()
for filename in listdir('Train\X'):
  trainf.append(filename)
for filename in listdir('Train\Y'):
  trainfe.append(filename)
for filename in listdir('Validation\X'):
  validf.append(filename)
for filename in listdir('Validation\Y'):
  validfe.append(filename)
trainf.sort()
trainfe.sort()
validf.sort()
validfe.sort()
#len(validf)
for i in range(len(trainf)):
  trainf[i] = 'Train\X'+'/'+ trainf[i]
for i in range(len(trainfe)):
  trainfe[i] = 'Train\Y'+'/'+ trainfe[i]
for i in range(len(validf)):
  validf[i] = 'Validation\X'+'/'+ validf[i]
for i in range(len(validfe)):
  validfe[i] = 'Validation\Y'+'/'+ validfe[i]
tf=len(trainf)
vf=len(validf)
#tf=500
#vf=100

In [3]:
import numpy as np
def eventroll_to_multihot_vector(eventroll):
    """
    Parameters
    ----------
    eventroll : np.array
        Eventroll matrix of shape=(11, 1000).
    
    Returns
    -------
    np.array
        A multihot vector of shape=(10,)
    """
    
    # findout active events:
    active_events = (eventroll.sum(axis=1) >= 0.5).astype('float')
    
    # remove silence class:
    return np.delete(active_events, 8)

In [4]:
import numpy as np
def eventroll_to_multihot_vector2(eventroll):
    """
    Parameters
    ----------
    eventroll : np.array
        Eventroll matrix of shape=(10, 1000).
    
    Returns
    -------
    np.array
        A multihot vector of shape=(10,)
    """
    
    # findout active events:
    active_events = (eventroll.sum(axis=1) >= 0.5).astype('float')
    
    # remove silence class:
    return active_events

In [5]:
train_features=[]
for i in range(tf):   #range(len(trainf))  range(50) 
  # m=np.load(trainf[i], allow_pickle=True)
  # n=np.load(trainfe[i], allow_pickle=True)
  m=np.load(trainf[i])
  n=np.load(trainfe[i])
  n=np.delete(n, 8, 0)
  
  #o=eventroll_to_multihot_vector(n)
  train_features.append([m[0].transpose(),n.transpose()])

In [6]:
valid_features=[]
for i in range(vf):   #range(len(validf))  range(10)
  # m=np.load(validf[i], allow_pickle=True)
  # n=np.load(validfe[i], allow_pickle=True)
  m=np.load(validf[i])
  n=np.load(validfe[i])
  n=np.delete(n, 8, 0)
  #o=eventroll_to_multihot_vector(n)
  valid_features.append([m[0].transpose(),n.transpose()])

In [7]:
train_features_df=pd.DataFrame(train_features,columns=['input','onehot'])
valid_features_df=pd.DataFrame(valid_features,columns=['input','onehot'])
X1=np.array(train_features_df['input'].tolist())
y1=np.array(train_features_df['onehot'].tolist())
X2=np.array(valid_features_df['input'].tolist())
y2=np.array(valid_features_df['onehot'].tolist())

In [8]:
X1=np.reshape(X1, (tf*1000, 64))
y1=np.reshape(y1, (tf*1000, 10))
X2=np.reshape(X2, (vf*1000, 64))
y2=np.reshape(y2, (vf*1000, 10))
y3=[0]*(vf*1000)
for i in range(vf*1000):
  for j in range(10):
    y3[i]=y3[i] + y2[i][j]*j
  y3[i]=int(y3[i])
y4=[0]*(tf*1000)
for i in range(tf*1000):
  for j in range(10):
    y4[i]=y4[i] + y1[i][j]*j
  y4[i]=int(y4[i])

In [9]:
# training a DescisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import multilabel_confusion_matrix
dtree_model = DecisionTreeClassifier(max_depth = 50).fit(X1, y1)
dtree_predictions = dtree_model.predict(X2)
#metrics.accuracy_score(y2, dtree_predictions)
dtree_prediction=np.reshape(dtree_predictions, (vf, 1000, 10)) 
y2=np.reshape(y2, (vf, 1000, 10))  
#predicting which sounds are present in the given file
onehotp2=list()
onehotgt2=list()
for i in range(vf):  #len(validf)=2000
  tp=eventroll_to_multihot_vector2(dtree_prediction[i].transpose());
  tq=eventroll_to_multihot_vector2(y2[i].transpose());
  onehotp2.append(tp)
  onehotgt2.append(tq)
#f1 scores calculation for different classes
P2=np.array(onehotp2)
Q2=np.array(onehotgt2)
from sklearn.metrics import f1_score
f2=[0,0,0,0,0,0,0,0,0,0]
for i in range(10):
  f2[i]=f1_score(onehotp2[:][i], onehotgt2[:][i], average='weighted')
print(f2)
#Finding the accuracy for individual classes and mean accuracy
from sklearn.metrics import accuracy_score
acm2=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
for i in range(10):
  accuracy = accuracy_score(Q2[:][i], P2[:][i])
  print('Accuracy ', 'for class ', "%d"%(i+1),': ' "%.2f" % (accuracy*100))
  acm2[i]=accuracy*100
acc2=0
for i in range(10):
  acc2=acc2 + acm2[i];
acc2=acc2/10
print('Mean accuracy is : ', "%0.2f"%(acc2))

[0.33333333333333337, 0.34949494949494947, 0.33333333333333337, 0.25050505050505045, 0.325, 0.4615384615384615, 0.4615384615384615, 0.4615384615384615, 0.18181818181818182, 0.33333333333333337]
Accuracy  for class  1 : 20.00
Accuracy  for class  2 : 30.00
Accuracy  for class  3 : 20.00
Accuracy  for class  4 : 30.00
Accuracy  for class  5 : 40.00
Accuracy  for class  6 : 30.00
Accuracy  for class  7 : 30.00
Accuracy  for class  8 : 30.00
Accuracy  for class  9 : 10.00
Accuracy  for class  10 : 20.00
Mean accuracy is :  26.00


In [10]:
#Mean f1 score calculation
f2m=0;
for i in range(10):
    f2m=f2m+f2[i]
f2m=f2m/10
print('Mean f1 score for Decision Tree Classifier is: ', "%0.2f"%(f2m*100))

Mean f1 score for Decision Tree Classifier is:  34.91


In [11]:
#Naive Bayes classifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB().fit(X1, y4)
gnb_predictions = gnb.predict(X2)
accuracy = gnb.score(X2, y3)
cm = confusion_matrix(y3, gnb_predictions)
y5=[[0]*(10)]*(vf*1000)
for i in range(vf*1000):
  for j in range(10):
    if gnb_predictions[i]==j:
      y5[i][j]=1
import numpy as np
y5=np.array(y5)
y2=np.reshape(y2, (vf, 1000, 10))
y5=np.reshape(y5, (vf, 1000, 10))
onehotp2=list()
onehotgt2=list()
for i in range(vf):  #len(validf)=2000
  tp=eventroll_to_multihot_vector2(y5[i].transpose());
  tq=eventroll_to_multihot_vector2(y2[i].transpose());
  onehotp2.append(tp)
  onehotgt2.append(tq)
#f1 scores calculation for different classes
P2=np.array(onehotp2)
Q2=np.array(onehotgt2)
from sklearn.metrics import f1_score
f2=[0,0,0,0,0,0,0,0,0,0]
for i in range(10):
  f2[i]=f1_score(onehotp2[:][i], onehotgt2[:][i], average='weighted')
print(f2)
#Finding the accuracy for individual classes and mean accuracy
from sklearn.metrics import accuracy_score
acm2=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
for i in range(10):
  accuracy = accuracy_score(Q2[:][i], P2[:][i])
  print('Accuracy ', 'for class ', "%d"%(i+1),': ' "%.2f" % (accuracy*100))
  acm2[i]=accuracy*100
acc2=0
for i in range(10):
  acc2=acc2 + acm2[i];
acc2=acc2/10
print('Mean accuracy is : ', "%0.2f"%(acc2))

[0.34949494949494947, 0.34949494949494947, 0.34949494949494947, 0.19999999999999998, 0.19999999999999998, 0.475, 0.475, 0.475, 0.19999999999999998, 0.34949494949494947]
Accuracy  for class  1 : 30.00
Accuracy  for class  2 : 30.00
Accuracy  for class  3 : 30.00
Accuracy  for class  4 : 20.00
Accuracy  for class  5 : 20.00
Accuracy  for class  6 : 40.00
Accuracy  for class  7 : 40.00
Accuracy  for class  8 : 40.00
Accuracy  for class  9 : 20.00
Accuracy  for class  10 : 30.00
Mean accuracy is :  30.00


In [12]:
#Mean f1 score calculation
f2m=0;
for i in range(10):
    f2m=f2m+f2[i]
f2m=f2m/10
print('Mean f1 score for Naive Bayes classifier is: ', "%0.2f"%(f2m*100))

Mean f1 score for Naive Bayes classifier is:  34.23


In [13]:
#KNN model
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 11).fit(X1, y4)
accuracy = knn.score(X2, y3)
print('Accuracy:', "%.2f"%(accuracy*100))
knn_pred=knn.predict(X2)
y5=[[0]*(10)]*(vf*1000)
for i in range(vf*1000):
  for j in range(10):
    if knn_pred[i]==j:
      y5[i][j]=1
import numpy as np
y5=np.array(y5)
y2=np.reshape(y2, (vf, 1000, 10))
y5=np.reshape(y5, (vf, 1000, 10))
onehotp2=list()
onehotgt2=list()
for i in range(vf):  #len(validf)=2000
  tp=eventroll_to_multihot_vector2(y5[i].transpose());
  tq=eventroll_to_multihot_vector2(y2[i].transpose());
  onehotp2.append(tp)
  onehotgt2.append(tq)
#f1 scores calculation for different classes
P2=np.array(onehotp2)
Q2=np.array(onehotgt2)
from sklearn.metrics import f1_score
f2=[0,0,0,0,0,0,0,0,0,0]
for i in range(10):
  f2[i]=f1_score(onehotp2[:][i], onehotgt2[:][i], average='weighted')
print(f2)
#Finding the accuracy for individual classes and mean accuracy
from sklearn.metrics import accuracy_score
acm2=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
for i in range(10):
  accuracy = accuracy_score(Q2[:][i], P2[:][i])
  print('Accuracy ', 'for class ', "%d"%(i+1),': ' "%.2f" % (accuracy*100))
  acm2[i]=accuracy*100
acc2=0
for i in range(10):
  acc2=acc2 + acm2[i];
acc2=acc2/10
print('Mean accuracy is : ', "%0.2f"%(acc2))

Accuracy: 76.46
[0.33333333333333337, 0.33333333333333337, 0.33333333333333337, 0.18181818181818182, 0.18181818181818182, 0.4615384615384615, 0.4615384615384615, 0.4615384615384615, 0.18181818181818182, 0.33333333333333337]
Accuracy  for class  1 : 20.00
Accuracy  for class  2 : 20.00
Accuracy  for class  3 : 20.00
Accuracy  for class  4 : 10.00
Accuracy  for class  5 : 10.00
Accuracy  for class  6 : 30.00
Accuracy  for class  7 : 30.00
Accuracy  for class  8 : 30.00
Accuracy  for class  9 : 10.00
Accuracy  for class  10 : 20.00
Mean accuracy is :  20.00


In [14]:
#Mean f1 score calculation
f2m=0;
for i in range(10):
    f2m=f2m+f2[i]
f2m=f2m/10
print('Mean f1 score for KNN is: ', "%0.2f"%(f2m*100))

Mean f1 score for KNN is:  32.63
